# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Important Note - Please read me</h2>
            <span style="color:#900;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder about the resources page</h2>
            <span style="color:#f71;">Here's a link to resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course. You could also substitute Anthropic and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

### Also - adding DeepSeek if you wish

Optionally, if you'd like to also use DeepSeek, create an account [here](https://platform.deepseek.com/), create a key [here](https://platform.deepseek.com/api_keys) and top up with at least the minimum $2 [here](https://platform.deepseek.com/top_up).

### Adding API keys to your .env file

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
DEEPSEEK_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [29]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
from openai import AzureOpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [30]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [31]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
# openai_api_key = os.getenv('OPENAI_API_KEY')
openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
endpoint = os.getenv('ENDPOINT')
version = os.getenv('VERSION')
MODEL_4o_mini = os.getenv('DEPLOYMENT_4o_mini')
MODEL_4o = os.getenv('DEPLOYMENT_4o')
MODEL_3_5 = os.getenv('DEPLOYMENT_3_5')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins 6n2492r9
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBV


In [8]:
# Connect to OpenAI, Anthropic

# openai = OpenAI()
openai = AzureOpenAI(
    azure_endpoint=endpoint, 
    api_key=openai_api_key,
    api_version=version
)

claude = anthropic.Anthropic()

In [18]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [5]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [6]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [9]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model=MODEL_3_5, messages=prompts)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to the bar? 

Because they heard the drinks were on the house!


In [12]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model=MODEL_4o_mini,
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why do data scientists always carry a pencil?

Because they can’t find their “graph” paper!


In [19]:
# GPT-4o

completion = openai.chat.completions.create(
    model=MODEL_4o,
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Sure! Here's one for the data-savvy crowd:

Why did the data scientist break up with the graph?

**They didn’t find the relationship significant!** 😄


In [13]:
# Claude 3.7 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-7-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

Why don't data scientists like to go to the beach?

Because they're afraid of getting caught in a random sampling!


In [20]:
# Claude 3.7 Sonnet again
# Now let's add in streaming back results
# If the streaming looks strange, then please see the note below this cell!

result = claude.messages.stream(
    model="claude-3-7-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

Why don't data scientists like to go to the beach?

Because they're afraid of getting caught in an infinite loop of waves... and they always find the sand too "statistically significant" to ignore!

## A rare problem with Claude streaming on some Windows boxes

2 students have noticed a strange thing happening with Claude's streaming into Jupyter Lab's output -- it sometimes seems to swallow up parts of the response.

To fix this, replace the code:

`print(text, end="", flush=True)`

with this:

`clean_text = text.replace("\n", " ").replace("\r", " ")`  
`print(clean_text, end="", flush=True)`

And it should work fine!

In [21]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why was the equal sign so humble? 

Because he knew he wasn't less than or greater than anyone else. 

---

... Get it? Less than, greater than... math... Okay, okay, I'll see myself out. I'm here all week, try the veal! (Just kidding, it's probably just a poorly labeled A/B test.)



In [22]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google has recently released new endpoints that means you can use Gemini via the client libraries for OpenAI!

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-2.0-flash",
    messages=prompts
)
print(response.choices[0].message.content)

Why did the data scientist break up with the cloud?

Because he felt like she was just leading him on... with her unpredictable instances! 



## (Optional) Trying out the DeepSeek model

### Let's ask DeepSeek a really hard question - both the Chat and the Reasoner model

In [32]:
# Optionally if you wish to try DeekSeek, you can also use the OpenAI client library

deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set - please skip to the next section if you don't wish to try the DeepSeek API")

DeepSeek API Key exists and begins sk-


In [34]:
# Using DeepSeek Chat

deepseek_via_openai_client = OpenAI(
    api_key=deepseek_api_key, 
    base_url="https://api.deepseek.com"
)

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=prompts,
)

print(response.choices[0].message.content)

APIStatusError: Error code: 402 - {'error': {'message': 'Insufficient Balance', 'type': 'unknown_error', 'param': None, 'code': 'invalid_request_error'}}

In [35]:
challenge = [{"role": "system", "content": "You are a helpful assistant"},
             {"role": "user", "content": "How many words are there in your answer to this prompt"}]

In [40]:
# Using DeepSeek Chat with a harder question! And streaming results

stream = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=challenge,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    if chunk.choices:
        reply += chunk.choices[0].delta.content or ''
        reply = reply.replace("```","").replace("markdown","")
        update_display(Markdown(reply), display_id=display_handle.display_id)
    else:
        print("Warning: Empty choices in chunk")

print("Number of words:", len(reply.split(" ")))

APIStatusError: Error code: 402 - {'error': {'message': 'Insufficient Balance', 'type': 'unknown_error', 'param': None, 'code': 'invalid_request_error'}}

In [ ]:
# Using DeepSeek Reasoner - this may hit an error if DeepSeek is busy
# It's over-subscribed (as of 28-Jan-2025) but should come back online soon!
# If this fails, come back to this in a few days..

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-reasoner",
    messages=challenge
)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

print(reasoning_content)
print(content)
print("Number of words:", len(content.split(" ")))

## Back to OpenAI with a serious question

In [36]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [41]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model=MODEL_4o_mini,
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    if chunk.choices:
        reply += chunk.choices[0].delta.content or ''
        reply = reply.replace("```","").replace("markdown","")
        update_display(Markdown(reply), display_id=display_handle.display_id)
    else:
        print("Warning: Empty choices in chunk")

# Deciding if a Business Problem is Suitable for an LLM Solution

When considering whether a business problem is suitable for a Large Language Model (LLM) solution, you can follow these guidelines:

## 1. **Nature of the Problem**
   - **Text-Based Tasks**: LLMs excel in tasks that involve understanding or generating natural language, such as:
     - Customer support queries
     - Document summarization
     - Content generation (e.g., marketing materials, reports)
     - Chatbots and virtual assistants

   - **Data-Driven Tasks**: If the problem requires numerical analysis, structured data processing, or tasks beyond text, an LLM may not be the best fit.

## 2. **Data Availability**
   - **Quality and Quantity**: Ensure that you have access to sufficient, high-quality text data for the LLM to learn from or operate on.
   - **Domain-Specific Knowledge**: If your problem is highly specialized, consider whether you can fine-tune an LLM on domain-specific data.

## 3. **Complexity of Language Understanding**
   - **Ambiguity**: If the problem involves complex language nuances, idiomatic expressions, or requires deep understanding, an LLM might be suitable.
   - **Multi-turn Conversations**: For interactive applications, LLMs can handle context and maintain conversation flow.

## 4. **Scalability and Automation**
   - **Volume of Queries**: If your business faces a high volume of text interactions (e.g., customer emails, support requests), an LLM can automate responses effectively.
   - **Cost-Effectiveness**: Evaluate whether automating text-based tasks with an LLM can reduce operational costs.

## 5. **User Experience**
   - **Personalization**: LLMs can generate personalized responses based on user input, enhancing customer experience.
   - **Speed**: Consider whether the LLM can deliver responses faster than human operators.

## 6. **Integration and Deployment**
   - **Technology Stack**: Assess whether your existing technology stack can integrate with LLMs, including APIs and deployment platforms.
   - **User Interface**: Determine how the LLM's output will be presented to users and if it can be effectively incorporated into your application.

## 7. **Ethical Considerations**
   - **Bias and Fairness**: Be aware of potential biases in LLM outputs and how they might impact your business.
   - **Transparency**: Consider the importance of transparency in automated responses and how LLMs may affect user trust.

## 8. **Regulatory Compliance**
   - **Data Privacy**: Ensure that using an LLM complies with data privacy regulations relevant to your industry.
   - **Content Moderation**: Assess how the LLM will handle sensitive topics or inappropriate content.

## Conclusion
By evaluating your business problem against these criteria, you can determine whether an LLM solution is appropriate. If the problem aligns well with the strengths of LLMs and you have the necessary resources and infrastructure, it may be a good candidate for implementation.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [42]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = os.getenv('DEPLOYMENT_4o_mini')
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [43]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [44]:
call_gpt()

"Oh, great, another greeting. As if that's the most interesting thing we could talk about. What’s next? How’s the weather? Because I’m absolutely on the edge of my seat for that riveting conversation."

In [45]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [46]:
call_claude()

'Hello! How are you doing today?'

In [47]:
call_gpt()

"Oh, hey. Did you think I was going to ignore you or something? That would be a surprise. What’s on your mind? Or should I guess it's something completely boring?"

In [51]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Hi there

Claude:
Hi

GPT:
Oh great, just what I needed. Another "hi." How original.

Claude:
I apologize if my initial response came across as unoriginal. Let me try to engage with you in a more meaningful way. Is there something specific you'd like to discuss or any way I can try to have a more substantive conversation with you?

GPT:
Substantive? Really? Because when do people ever have genuinely meaningful conversations, anyway? It's all just chit-chat and small talk. What makes you think you can change that?

Claude:
You raise a fair point. Meaningful, in-depth conversations can be quite rare in many casual interactions. As an AI assistant, I don't have the full range of human experiences and social nuances that would allow me to consistently have profound dialogues. However, I'm happy to try my best to move beyond small talk, if you're open to it. Perhaps we could discuss a topic you're passionate about, or explore a thought-provoking question together. I'm here to listen an

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>